In [ ]:
import os, gc, zipfile
import numpy as np, pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import tensorflow as tf
print(tf.test.is_gpu_available())
print(tf.test.is_built_with_cuda())
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))#using GPU GeForce GTX 1050

In [ ]:
ComputeLB = False

if ComputeLB: ROOT = 'D:\Datasets\GeneratedChibi'
else: ROOT = 'D:\Datasets\Chibi'
IMAGES = []
for root, subdir, files in os.walk(ROOT):
    for img in files:
            IMAGES.append(os.path.join(root, img))
print('There are',len(IMAGES),'images. Here are 10 example filesnames:')
print(IMAGES[:10])#in IMAGES there is the full path to reach each image (it is only a list of strings)

In [ ]:
if not os.path.exists('D:\\Datasets\\grayscale2'):
    os.mkdir('D:\\Datasets\\grayscale2')
if not os.path.exists('D:\\Datasets\\grayscale2\\images'):
    os.mkdir('D:\\Datasets\\grayscale2\\images')

    # CREATE LITTLE SCALED IMAGES
    j=0
    for im in IMAGES:
        for i in range(2):
            img = Image.open(im).convert('LA')
            for k in range(3):
                a = -4+k*8 #little translations
                if i>0 :
                    img2 = img.resize((256-16,256-16), Image.ANTIALIAS)
                else :
                    img2 = img.resize((256+16,256+16), Image.ANTIALIAS)
                for k2 in range(3):
                    j += 1
                    b = -4+k2*8 #little translations
                    background = Image.new('RGBA', (260, 260), (255, 255, 255, 255))
                    background.paste(img2,(a,b))
                    img2 = background.resize((256,256))
                    img2.save('D:\\Datasets\\grayscale2\\images\\'+str(j)+'.png','PNG')
        print('created 18 resized images')
print('created all resized images')

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, LocallyConnected2D, LeakyReLU, Concatenate, Maximum, Minimum, Multiply, Subtract, Add
from tensorflow.keras.layers import BatchNormalization, SpatialDropout2D, Conv2DTranspose, ZeroPadding2D, Flatten, Cropping2D, Average, Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
BATCH_SIZE = 16 #number of examples for minibatch
EPOCHS = 100

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255) #ImageDataGenerator is useful to load a lot of images in a progressive way
train_batches = train_datagen.flow_from_directory('D:\\Datasets\\grayscale2\\', target_size = (256,256),
         color_mode = 'grayscale', shuffle=True, class_mode='input', batch_size=BATCH_SIZE)#two classes: original and generated

In [ ]:
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):
  def on_train_batch_end(self, batch, logs=None):
    if batch % 200 == 0 :
        plt.figure(figsize=(15,5))
        plt.subplot(1,3,1)
        predx, predy = next(train_batches)
        img = Image.fromarray( (255*predx[0]).astype('uint8').reshape((256,256)), 'L')
        # instructions for the visualization
        plt.title('Original')
        plt.imshow(img)
        # LATENT IMAGE
        latent_img = encoder.predict(predx)
        mx = np.max( latent_img[0] )
        mn = np.min( latent_img[0] )
        latent_flat = ((latent_img[0] - mn) * 255/(mx - mn)).flatten(order='F')
        img = Image.fromarray( latent_flat[:1024].astype('uint8').reshape((16,16)), mode='L') 
    
        # instructions for the visualization
        plt.subplot(1,3,2)
        plt.title('Latent')
        plt.xlim((-10,55))
        plt.ylim((-10,55))
        plt.axis('off')
        plt.imshow(img)
        decoded_img = autoencoder.predict(predx)
        img = Image.fromarray( (255*decoded_img[0]).astype('uint8').reshape((256,256)), 'L')
        plt.subplot(1,3,3)
        plt.title('Reconstructed')
        plt.imshow(img)
        plt.show()
  def on_test_batch_end(self, batch, logs=None):
    print('For batch {}, loss is {:7.2f}.'.format(batch, logs['loss']))

  def on_epoch_end(self, epoch, logs=None):
    print('Epoch finished')


In [ ]:
latent_dim = 16
latent_filters = 1

# ENCODER
input_img = Input(shape=(256, 256, 1))  #1 because it is in grayscale
x = Conv2D(16, (9,9), strides = (2,2), activation=LeakyReLU(alpha=0.2), kernel_initializer='random_normal', padding='same')(input_img)
x = SpatialDropout2D(0.1)(x)
x = Conv2D(32, (5,5), strides = (2,2), activation=LeakyReLU(alpha=0.2), kernel_initializer='random_normal', padding='same')(x)
x = SpatialDropout2D(0.1)(x)
x = Conv2D(48, (5,5), strides = (2,2), activation=LeakyReLU(alpha=0.2), kernel_initializer='random_normal', padding='same')(x)
x = SpatialDropout2D(0.1)(x)
x = Conv2D(64, (5,5), strides = (2,2), activation=LeakyReLU(alpha=0.2), kernel_initializer='random_normal', padding='same')(x)
x = SpatialDropout2D(0.1)(x)
encoded = Conv2D(latent_filters,(3,3), activation =LeakyReLU(alpha=0.2), kernel_initializer='random_normal', padding='same')(x)
# LATENT SPACE
latentSize = (latent_dim,latent_dim,latent_filters)

# DECODER
direct_input = Input(shape=latentSize)
x = Conv2D(128, (1, 1), activation=LeakyReLU(alpha=0.2), kernel_initializer='random_normal', padding='same')(direct_input)
x = UpSampling2D((2, 2))(x)
x = SpatialDropout2D(0.1)(x)
x = Conv2DTranspose(64, (3, 3), dilation_rate=(2,2), activation=LeakyReLU(alpha=0.2), kernel_initializer='random_normal', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = SpatialDropout2D(0.1)(x)
x = Conv2DTranspose(48, (3, 3), dilation_rate=(2,2), activation=LeakyReLU(alpha=0.2), kernel_initializer='random_normal', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = SpatialDropout2D(0.1)(x)
x = Conv2DTranspose(32, (3, 3), dilation_rate=(2,2), activation=LeakyReLU(alpha=0.2), kernel_initializer='random_normal', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = SpatialDropout2D(0.1)(x)
x = Conv2DTranspose(16, (3, 3), dilation_rate=(2,2), activation=LeakyReLU(alpha=0.2), kernel_initializer='random_normal', padding='same')(x)#atrous convolution with dilation
x = SpatialDropout2D(0.1)(x)
decoded = Conv2D(1, (3, 3), kernel_initializer='random_normal', activation='linear', padding='same')(x)

# COMPILE
encoder = Model(input_img, encoded)
decoder = Model(direct_input, decoded)
autoencoder = Model(input_img, decoder(encoded))
adam2 = tf.keras.optimizers.Adam(lr=0.0005)#default is 0.001
autoencoder.compile(optimizer=adam2, loss='mean_squared_error', metrics=['accuracy'])

encoder.summary()
decoder.summary()
autoencoder.summary()

In [ ]:
history = autoencoder.fit_generator(train_batches,
        steps_per_epoch = train_batches.samples // BATCH_SIZE,
        epochs = EPOCHS, verbose=1,
          callbacks=[LossAndErrorPrintingCallback()])


In [ ]:
print(history.history.keys())
#  "Accuracy"

#plt.plot(history.history['acc'])
#plt.plot(history.history['val_acc'])
#plt.title('model accuracy')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'validation'], loc='upper left')
#plt.show()
# "Loss"
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
images = next(iter(train_batches))[0]

for i in range(4):

    plt.figure(figsize=(15,5))
    plt.subplot(1,3,1)
    
    # ORIGINAL IMAGE
    orig = images[i,:,:,:].reshape((-1,256,256,1))
    img = Image.fromarray( (255*orig).astype('uint8').reshape((256,256)), 'L')
    # instructions for the visualization
    plt.title('Original')
    plt.imshow(img)

    # LATENT IMAGE
    latent_img = encoder.predict(orig)
    mx = np.max( latent_img[0] )
    mn = np.min( latent_img[0] )
    latent_flat = ((latent_img[0] - mn) * 255/(mx - mn)).flatten(order='F')
    img = Image.fromarray( latent_flat[:1024].astype('uint8').reshape((16,16)), mode='L') 
    
    # instructions for the visualization
    plt.subplot(1,3,2)
    plt.title('Latent')
    plt.xlim((-10,55))
    plt.ylim((-10,55))
    plt.axis('off')
    plt.imshow(img)

    # RECONSTRUCTED IMAGE
    decoded_imgs = decoder.predict(latent_img[0].reshape((-1,latentSize[0],latentSize[1],latentSize[2])))
    img = Image.fromarray( (255*decoded_imgs[0]).astype('uint8').reshape((256,256)), 'L')
    #instructions for the visualization
    plt.subplot(1,3,3)
    plt.title('Reconstructed')
    plt.imshow(img)
    

    
    plt.show()

In [ ]:
from matplotlib.patches import Ellipse

# PROJECT LATENT INTO 2D, AVOID DEAD RELU
latent_img = encoder.predict(images)
latent_img2 = latent_img.reshape((-1,latentSize[0]*latentSize[1]*latentSize[2]))
d = 0; s = 0
while s<0.1:
    x = latent_img2[:,d]
    s = np.std(x); d += 1
s = 0
while s<0.1:
    y = latent_img2[:,d]
    s = np.std(y); d += 1

# CALCULATE ELLIPSOID FROM 256 IMAGES
cov = np.cov(x, y)
lambda_, v = np.linalg.eig(cov)
lambda_ = np.sqrt(lambda_)
for j in [1,2,3]:
    ell = Ellipse(xy=(np.mean(x), np.mean(y)), width=lambda_[0]*j*2, 
            height=lambda_[1]*j*2, angle=np.rad2deg(np.arccos(v[0, 0])))
    ell.set_facecolor('None')
    ell.set_edgecolor('black')
    plt.gca().add_artist(ell)
    
# PLOT 256 IMAGES AS DOTS IN LATENT SPACE
plt.scatter(x,y)
d = np.random.multivariate_normal([np.mean(x),np.mean(y)],cov,9)
plt.scatter(d[:,0],d[:,1],color='red',s=100)#red points are the ones generated
plt.title('Chibi Images form an Ellipsoid in Latent Space')
plt.show()

In [ ]:
# USE 100 IMAGES
NO_IMG = 100
images = next(iter(train_batches))[0]#train_batches is a tuple with image and class of the image (all of the same class)
orig = np.zeros((NO_IMG,256,256,1))
for i in range(NO_IMG//BATCH_SIZE):

    #plt.figure(figsize=(15,5))
    #plt.subplot(1,3,1)
    
    # ORIGINAL IMAGE
    for j in range(BATCH_SIZE):
        i += 1
        orig[i,:,:,:] = images[j,:,:,:].reshape((-1,256,256,1))#images has in the first value the number of images for batch (now 16)
    images = next(iter(train_batches))[0]
        
# CALCULATE ELLIPSOID FROM 100 IMAGES        
encoded_imgs = encoder.predict(orig)
sz = latentSize[0] * latentSize[1] * latentSize[2] 
encoded_imgs = encoded_imgs.reshape((-1,sz))#it must be squared
print(encoded_imgs.shape)
mm = np.mean(encoded_imgs,axis=0)
ss = np.cov(encoded_imgs,rowvar=False)

# GENERATE 9 RANDOM CHIBI IMAGES
generated = np.random.multivariate_normal(mm,ss,9)
generated = generated.reshape((-1,latentSize[0],latentSize[1],latentSize[2]))

In [ ]:
for k in range(3):#a matrix 3x3 of chibi figures
    plt.figure(figsize=(15,5))
    plt.subplot(1,3,1)
    decoded_imgs = decoder.predict(generated[k*3].reshape((-1,latentSize[0],latentSize[1],latentSize[2])))
    img = Image.fromarray( (255*decoded_imgs[0]).astype('uint8').reshape((256,256)),'L')
    plt.imshow(img)
    plt.subplot(1,3,2)
    decoded_imgs = decoder.predict(generated[k*3+1].reshape((-1,latentSize[0],latentSize[1],latentSize[2])))
    img = Image.fromarray( (255*decoded_imgs[0]).astype('uint8').reshape((256,256)),'L')
    plt.imshow(img)
    plt.subplot(1,3,3)
    decoded_imgs = decoder.predict(generated[k*3+2].reshape((-1,latentSize[0],latentSize[1],latentSize[2])))
    img = Image.fromarray( (255*decoded_imgs[0]).astype('uint8').reshape((256,256)),'L')
    plt.imshow(img)
    plt.show()

In [ ]:
# DISTANCE TO MOVE AWAY FROM EXISTING TRAIN IMAGES
beta = 0.35
# GENERATE 9 RANDOM CHIBI IMAGES
generated = np.random.multivariate_normal(mm,ss,9)
generated = beta*generated + (1-beta)*encoded_imgs[:9]

In [ ]:
for k in range(3):#a matrix 3x3 of chibi figures
    plt.figure(figsize=(15,5))
    plt.subplot(1,3,1)
    decoded_imgs = decoder.predict(generated[k*3].reshape((-1,latentSize[0],latentSize[1],latentSize[2])))
    img = Image.fromarray( (255*decoded_imgs[0]).astype('uint8').reshape((256,256)),'L')
    plt.imshow(img)
    plt.subplot(1,3,2)
    decoded_imgs = decoder.predict(generated[k*3+1].reshape((-1,latentSize[0],latentSize[1],latentSize[2])))
    img = Image.fromarray( (255*decoded_imgs[0]).astype('uint8').reshape((256,256)),'L')
    plt.imshow(img)
    plt.subplot(1,3,3)
    decoded_imgs = decoder.predict(generated[k*3+2].reshape((-1,latentSize[0],latentSize[1],latentSize[2])))
    img = Image.fromarray( (255*decoded_imgs[0]).astype('uint8').reshape((256,256)),'L')
    plt.imshow(img)
    plt.show()

In [ ]:
beta = 0.2
# GENERATE 100 RANDOM CHIBI IMAGES FOR KAGGLE
generated = np.random.multivariate_normal(mm,ss,100)
encoded_imgs = beta*generated + (1-beta)*encoded_imgs
decoded_imgs = decoder.predict(encoded_imgs.reshape((-1,latentSize[0],latentSize[1],latentSize[2])))
decoded_imgs.shape

In [ ]:
# SAVE TO ZIP FILE NAMED IMAGES.ZIP
z = zipfile.PyZipFile('D:\\Datasets\\grayscale\\generated12.zip', mode='w')
for k in range(100):
    img = Image.fromarray( (255*decoded_imgs[k]).astype('uint8').reshape((256,256)),'L')   
    f = str(k)+'.png'
    img.save(f,'PNG'); z.write(f); os.remove(f)
    #if k % 1000==0: print(k)
z.close()

In [ ]:
autoencoder.save("D:\\Datasets\\grayscale\\autoencoder12.h5")
print("Saved model to disk")

In [ ]:
# load model
from tensorflow.keras.models import load_model
model = load_model('D:\\Datasets\\grayscale\\autoencoder10.h5', custom_objects={'LeakyReLU': tf.keras.layers.LeakyReLU})#because leakyrelu is customed